<a href="https://colab.research.google.com/github/bcdanl/320-code/blob/main/danl_320_script_2025_0219.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, col, pow, mean, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.master("local[*]").getOrCreate()

# 1. Read CSV data from URL
df_pd = pd.read_csv('https://bcdanl.github.io/data/home_sales_nyc.csv')
sale_df = spark.createDataFrame(df_pd)
sale_df.show()

+---------+-------+------------+--------------------+-----------------------+--------+----------+-----------------+----------------+-----------+----------------+-----------------+----------+---------+---+
|sale.date|borough|borough_name|        neighborhood|building_class_category|zip_code|sale_price|residential_units|commercial_units|total_units|land_square_feet|gross_square_feet|year_built|sale_year|age|
+---------+-------+------------+--------------------+-----------------------+--------+----------+-----------------+----------------+-----------+----------------+-----------------+----------+---------+---+
|  8/31/18|      2|       Bronx|CASTLE HILL/UNION...|   01 ONE FAMILY DWE...|   10473|    492525|                1|               0|          1|            2250|             1540|      1960|     2018| 58|
|  8/31/18|      2|       Bronx|           SOUNDVIEW|   01 ONE FAMILY DWE...|   10473|    105000|                1|               0|          1|            3980|             1025| 

# Split data into training-test

In [33]:
dtrain, dtest = sale_df.randomSplit([0.6, 0.4], seed = 123)


In [4]:
dtrain.count()

7553

In [5]:
dtest.count()

5115

In [6]:
sale_df.count()

12668

# Model 1

## Assemble predictors into one column

In [34]:
# Now assemble predictors using the renamed column
assembler1 = VectorAssembler(
    inputCols=["gross_square_feet"],
    outputCol="predictors")

In [8]:
type(assembler1)

pyspark.ml.feature.VectorAssembler

In [10]:
assembler1.transform(dtrain).show()

+---------+-------+------------+--------------------+-----------------------+--------+----------+-----------------+----------------+-----------+----------------+-----------------+----------+---------+---+----------+
|sale.date|borough|borough_name|        neighborhood|building_class_category|zip_code|sale_price|residential_units|commercial_units|total_units|land_square_feet|gross_square_feet|year_built|sale_year|age|predictors|
+---------+-------+------------+--------------------+-----------------------+--------+----------+-----------------+----------------+-----------+----------------+-----------------+----------+---------+---+----------+
|  2/27/18|      2|       Bronx|MORRIS PARK/VAN NEST|   01 ONE FAMILY DWE...|   10462|    349830|                1|               0|          1|            2500|             1742|      1915|     2018|103|  [1742.0]|
|  2/27/18|      2|       Bronx|         THROGS NECK|   01 ONE FAMILY DWE...|   10465|    570000|                1|               0|    

## Add the column of "predictors" to dtrain and dtest

In [35]:
dtrain1 = assembler1.transform(dtrain) # training data
dtest1  = assembler1.transform(dtest)  # test data # for prediction purpose

dtrain1.select("predictors", "sale_price").show()
dtest1.select("predictors", "sale_price").show()

+----------+----------+
|predictors|sale_price|
+----------+----------+
|  [1742.0]|    349830|
|  [2165.0]|    570000|
|  [1116.0]|    410000|
|  [1120.0]|    260000|
|  [2304.0]|    543556|
|  [1830.0]|    610000|
|  [4200.0]|   1650000|
|  [1446.0]|    905000|
|  [1445.0]|   1150000|
|   [978.0]|    850000|
|  [1444.0]|    980000|
|  [1510.0]|   1130000|
|  [1428.0]|    425000|
|  [1712.0]|    860000|
|  [1200.0]|   1020000|
|  [2363.0]|   1200000|
|  [1197.0]|    427000|
|  [1080.0]|    423000|
|  [1392.0]|    325000|
|  [1400.0]|    597000|
+----------+----------+
only showing top 20 rows

+----------+----------+
|predictors|sale_price|
+----------+----------+
|  [1080.0]|    495000|
|  [2320.0]|    575000|
|  [2844.0]|   1700000|
|  [1696.0]|    650000|
|  [2000.0]|   1350000|
|  [2300.0]|   1500000|
|  [1314.0]|    940000|
|  [2452.0]|    550000|
|  [1125.0]|    820000|
|  [1690.0]|    235000|
|  [2163.0]|    695000|
|  [1870.0]|    590000|
|  [2052.0]|    715000|
|  [1728.0]|  

## Fit Linear Regression Model on Training Data

In [36]:
model1 = (
    LinearRegression(
        featuresCol= "predictors",
        labelCol= "sale_price"
    ).fit(dtrain1)
)

In [16]:
model1.intercept

-28248.33125480523

In [17]:
model1.coefficients


DenseVector([456.5836])

In [18]:
model1.summary.coefficientStandardErrors


[9.1585344019424, 14842.026122561703]

In [19]:
model1.summary.rootMeanSquaredError


459852.1900902878

In [20]:
model1.summary.r2

0.24763531485957802

## Making a Regression Table

In [21]:
import numpy as np
import scipy.stats as stats
from tabulate import tabulate

def regression_table(model, assembler):
    """
    Creates a formatted regression table from a fitted LinearRegression model and its VectorAssembler,
    and inserts a dashed horizontal line after the Intercept row. The table includes separate columns
    for the 95% confidence interval lower and upper bounds for each coefficient (computed at the 5% significance level)
    and an "Observations" row (using model.summary.numInstances) above the R² row.
    The RMSE row is placed as the last row.

    The columns are ordered as:
        Metric | Value | Significance | Std. Error | p-value | 95% CI Lower | 95% CI Upper

    For the "Value", "Std. Error", "95% CI Lower", and "95% CI Upper" columns, commas are inserted every three digits,
    with 3 decimal places (except for Observations which is formatted as an integer with commas).

    Parameters:
        model: A fitted LinearRegression model (with a .summary attribute).
        assembler: The VectorAssembler used to assemble the features for the model.

    Returns:
        A formatted string containing the regression table.
    """
    # Extract coefficients and standard errors as NumPy arrays
    coeffs = model.coefficients.toArray()
    std_errors_all = np.array(model.summary.coefficientStandardErrors)

    # Check if the intercept's standard error is included (one extra element)
    if len(std_errors_all) == len(coeffs) + 1:
        intercept_se = std_errors_all[0]
        std_errors = std_errors_all[1:]
    else:
        intercept_se = None
        std_errors = std_errors_all

    # Compute t-statistics for feature coefficients (t = beta / SE(beta))
    # t_stats = coeffs / std_errors
    t_stats = model.summary.tValues

    # Degrees of freedom: number of instances minus number of predictors minus 1 (for intercept)
    df = model.summary.numInstances - len(coeffs) - 1

    # Compute the t-critical value for a 95% confidence interval (two-tailed, 5% significance)
    t_critical = stats.t.ppf(0.975, df)

    # Compute two-tailed p-values for each feature coefficient
    # p_values = [2 * (1 - stats.t.cdf(np.abs(t), df)) for t in t_stats]
    p_values = model.summary.pValues

    # Function to assign significance stars based on p-value
    def significance_stars(p):
        if p < 0.01:
            return "***"
        elif p < 0.05:
            return "**"
        elif p < 0.1:
            return "*"
        else:
            return ""

    # Build the table rows.
    # Order: Metric, Value, Significance, Std. Error, p-value, 95% CI Lower, 95% CI Upper.
    table = []
    for feature, beta, se, p in zip(assembler.getInputCols(), coeffs, std_errors, p_values):
        ci_lower = beta - t_critical * se
        ci_upper = beta + t_critical * se
        table.append([
            "Beta: " + feature,       # Metric name
            beta,                     # Beta estimate (Value)
            significance_stars(p),    # Significance stars
            se,                       # Standard error
            p,                        # p-value
            ci_lower,                 # 95% CI lower bound
            ci_upper                  # 95% CI upper bound
        ])

    # Compute and add the intercept row with its SE, p-value, significance, and CI (if available)
    if intercept_se is not None:
        intercept_t = model.intercept / intercept_se
        intercept_p = 2 * (1 - stats.t.cdf(np.abs(intercept_t), df))
        intercept_sig = significance_stars(intercept_p)
        ci_intercept_lower = model.intercept - t_critical * intercept_se
        ci_intercept_upper = model.intercept + t_critical * intercept_se
    else:
        intercept_se = ""
        intercept_p = ""
        intercept_sig = ""
        ci_intercept_lower = ""
        ci_intercept_upper = ""

    table.append([
        "Intercept",
        model.intercept,
        intercept_sig,
        intercept_se,
        intercept_p,
        ci_intercept_lower,
        ci_intercept_upper
    ])

    # Append overall model metrics:
    # Insert an Observations row using model.summary.numInstances,
    # then an R² row, and finally the RMSE row as the last row.
    table.append(["Observations", model.summary.numInstances, "", "", "", "", ""])
    table.append(["R²", model.summary.r2, "", "", "", "", ""])
    table.append(["RMSE", model.summary.rootMeanSquaredError, "", "", "", "", ""])

    # Format the table.
    # For the "Value" (index 1), "Std. Error" (index 3), "95% CI Lower" (index 5), and "95% CI Upper" (index 6) columns,
    # format with commas and 3 decimal places, except for Observations which should be an integer with commas.
    # For the p-value (index 4), format to 3 decimal places.
    formatted_table = []
    for row in table:
        formatted_row = []
        for i, item in enumerate(row):
            if row[0] == "Observations" and i == 1 and isinstance(item, (int, float, np.floating)) and item != "":
                # Format Observations as integer with commas, no decimals.
                formatted_row.append(f"{int(item):,}")
            elif isinstance(item, (int, float, np.floating)) and item != "":
                if i in [1, 3, 5, 6]:
                    formatted_row.append(f"{item:,.3f}")
                elif i == 4:
                    formatted_row.append(f"{item:.3f}")
                else:
                    formatted_row.append(f"{item:.3f}")
            else:
                formatted_row.append(item)
        formatted_table.append(formatted_row)

    # Generate the table string using tabulate.
    table_str = tabulate(
        formatted_table,
        headers=["Metric", "Value", "Sig.", "Std. Error", "p-value", "95% CI Lower", "95% CI Upper"],
        tablefmt="pretty",
        colalign=("left", "right", "center", "right", "right", "right", "right")
    )

    # Insert a dashed line after the Intercept row for clarity.
    lines = table_str.split("\n")
    dash_line = '-' * len(lines[0])
    for i, line in enumerate(lines):
        if "Intercept" in line and not line.strip().startswith('+'):
            lines.insert(i+1, dash_line)
            break

    return "\n".join(lines)

# Example usage:
# print(regression_table(lr_model, assembler))

In [22]:
print( regression_table(model1, assembler1) )

+-------------------------+-------------+------+------------+---------+--------------+--------------+
| Metric                  |       Value | Sig. | Std. Error | p-value | 95% CI Lower | 95% CI Upper |
+-------------------------+-------------+------+------------+---------+--------------+--------------+
| Beta: gross_square_feet |     456.584 | ***  | 14,842.026 |   0.000 |  -28,637.917 |   29,551.084 |
| Intercept               | -28,248.331 | ***  |      9.159 |   0.000 |  -28,266.285 |  -28,230.378 |
-----------------------------------------------------------------------------------------------------
| Observations            |       7,553 |      |            |         |              |              |
| R²                      |       0.248 |      |            |         |              |              |
| RMSE                    | 459,852.190 |      |            |         |              |              |
+-------------------------+-------------+------+------------+---------+-----------

## Making a Prediction

In [38]:
# making a prediction
dtest1  = model1.transform(dtest1)


In [25]:
dtest1.show()

+---------+-------+------------+--------------------+-----------------------+--------+----------+-----------------+----------------+-----------+----------------+-----------------+----------+---------+---+----------+------------------+
|sale.date|borough|borough_name|        neighborhood|building_class_category|zip_code|sale_price|residential_units|commercial_units|total_units|land_square_feet|gross_square_feet|year_built|sale_year|age|predictors|        prediction|
+---------+-------+------------+--------------------+-----------------------+--------+----------+-----------------+----------------+-----------+----------------+-----------------+----------+---------+---+----------+------------------+
|  2/27/18|      2|       Bronx|MORRIS PARK/VAN NEST|   01 ONE FAMILY DWE...|   10462|    495000|                1|               0|          1|            2500|             1080|      1905|     2018|113|  [1080.0]|464861.95271934435|
|  2/27/18|      2|       Bronx|PELHAM PARKWAY NORTH|   01 O

# Model 2 - Multivariate Linear Regression

In [39]:
assembler2 = VectorAssembler(
                    inputCols=["gross_square_feet", "age"],
                    outputCol="predictors")

In [40]:
dtrain2 = assembler2.transform(dtrain)
dtest2  = assembler2.transform(dtest)

In [41]:
model2 = (
    LinearRegression(
                featuresCol="predictors",
                labelCol="sale_price")
        .fit(dtrain2)
)
dtest2 = model2.transform(dtest2)

print(regression_table(model2, assembler2))

+-------------------------+--------------+------+------------+---------+--------------+--------------+
| Metric                  |        Value | Sig. | Std. Error | p-value | 95% CI Lower | 95% CI Upper |
+-------------------------+--------------+------+------------+---------+--------------+--------------+
| Beta: gross_square_feet |      463.107 | ***  |    192.136 |   0.000 |       86.467 |      839.748 |
| Beta: age               |    2,429.358 | ***  | 20,938.514 |   0.000 |  -38,615.955 |   43,474.670 |
| Intercept               | -216,925.023 | ***  |      9.078 |   0.000 | -216,942.819 | -216,907.227 |
------------------------------------------------------------------------------------------------------
| Observations            |        7,553 |      |            |         |              |              |
| R²                      |        0.263 |      |            |         |              |              |
| RMSE                    |  455,059.540 |      |            |         | 

In [46]:
sale_df.select("borough_name").distinct().show()

+-------------+
| borough_name|
+-------------+
|       Queens|
|     Brooklyn|
|Staten Island|
|    Manhattan|
|        Bronx|
+-------------+



# Dummy Variable Linear Regression

In [47]:
def add_dummy_variables(var_name, reference_level):
    """
    Creates dummy variables for the specified column in the global DataFrames dtrain and dtest.
    If the variable contains boolean values, they are converted to strings.

    Parameters:
        var_name (str): The name of the categorical column (e.g., "borough_name").
        reference_level (int): The index (after sorting the categories) of the category
                               to be used as the reference (dummy omitted).

    Returns:
        dummy_cols (list): List of dummy column names to be used in subsequent analysis
                           (the reference category dummy is omitted).
        ref_category (str): The category value that has been chosen as the reference.
    """
    global dtrain, dtest

    # Get distinct categories from the training set.
    categories = dtrain.select(var_name).distinct().rdd.flatMap(lambda x: x).collect()

    # If any category is a boolean, convert all categories to string.
    categories = [str(c) if isinstance(c, bool) else c for c in categories]

    # Sort the categories.
    categories = sorted(categories)

    # Check that reference_level is valid
    if reference_level < 0 or reference_level >= len(categories):
        raise ValueError(f"reference_level must be between 0 and {len(categories) - 1}")

    # Determine the reference category based on the reference level
    ref_category = categories[reference_level]
    print("Reference category (dummy omitted):", ref_category)

    # Create dummy variables for all categories
    for cat in categories:
        # Always convert the category value to string and replace spaces.
        dummy_col_name = var_name + "_" + str(cat).replace(" ", "_")
        dtrain = dtrain.withColumn(dummy_col_name, when(col(var_name) == cat, 1).otherwise(0))
        dtest  = dtest.withColumn(dummy_col_name, when(col(var_name) == cat, 1).otherwise(0))

    # Build a list of dummy column names for regression (exclude reference category)
    dummy_cols = [var_name + "_" + str(cat).replace(" ", "_") for cat in categories if cat != ref_category]

    return dummy_cols, ref_category

In [48]:
# Distinct categories:
(
    dtrain
    .select("borough_name")
    .distinct()
    .orderBy("borough_name")
    .show()
)

+-------------+
| borough_name|
+-------------+
|        Bronx|
|     Brooklyn|
|    Manhattan|
|       Queens|
|Staten Island|
+-------------+



In [49]:
# Number of categories
(
    dtrain
    .select("borough_name")
    .distinct()
    .count()
)

5

In [50]:
# 0: Bronx, 1: Brooklyn, 2: Manhattan, 3: Queens, 4: Staten Island
dummy_cols, ref_category = add_dummy_variables("borough_name", 2)
# To see dummy variables
dtrain.select(['borough_name'] + dummy_cols).show()

Reference category (dummy omitted): Manhattan
+------------+------------------+---------------------+-------------------+--------------------------+
|borough_name|borough_name_Bronx|borough_name_Brooklyn|borough_name_Queens|borough_name_Staten_Island|
+------------+------------------+---------------------+-------------------+--------------------------+
|       Bronx|                 1|                    0|                  0|                         0|
|       Bronx|                 1|                    0|                  0|                         0|
|    Brooklyn|                 0|                    1|                  0|                         0|
|    Brooklyn|                 0|                    1|                  0|                         0|
|    Brooklyn|                 0|                    1|                  0|                         0|
|    Brooklyn|                 0|                    1|                  0|                         0|
|    Brooklyn|             

In [51]:
dtrain.select(['borough_name'] + dummy_cols).filter(col('borough_name') == "Manhattan").show()

+------------+------------------+---------------------+-------------------+--------------------------+
|borough_name|borough_name_Bronx|borough_name_Brooklyn|borough_name_Queens|borough_name_Staten_Island|
+------------+------------------+---------------------+-------------------+--------------------------+
|   Manhattan|                 0|                    0|                  0|                         0|
|   Manhattan|                 0|                    0|                  0|                         0|
|   Manhattan|                 0|                    0|                  0|                         0|
|   Manhattan|                 0|                    0|                  0|                         0|
|   Manhattan|                 0|                    0|                  0|                         0|
|   Manhattan|                 0|                    0|                  0|                         0|
|   Manhattan|                 0|                    0|                  

In [52]:
dummy_cols

['borough_name_Bronx',
 'borough_name_Brooklyn',
 'borough_name_Queens',
 'borough_name_Staten_Island']

In [53]:
# Define the list of predictor columns:
# Two numeric predictors plus the dummy variables (excluding the reference dummy)
conti_cols = ["gross_square_feet", "age"]
assembler_predictors = conti_cols + dummy_cols
assembler_predictors

['gross_square_feet',
 'age',
 'borough_name_Bronx',
 'borough_name_Brooklyn',
 'borough_name_Queens',
 'borough_name_Staten_Island']

In [54]:

assembler3 = VectorAssembler(
    inputCols = assembler_predictors,
    outputCol = "predictors"
)
dtrain3 = assembler3.transform(dtrain)
dtest3  = assembler3.transform(dtest)
model3 = LinearRegression(featuresCol="predictors", labelCol="sale_price").fit(dtrain3)
dtest3 = model3.transform(dtest3)

# For model3 and assembler3:
print(regression_table(model3, assembler3))

+----------------------------------+----------------+------+------------+---------+----------------+----------------+
| Metric                           |          Value | Sig. | Std. Error | p-value |   95% CI Lower |   95% CI Upper |
+----------------------------------+----------------+------+------------+---------+----------------+----------------+
| Beta: gross_square_feet          |        391.989 | ***  |    204.243 |   0.000 |         -8.384 |        792.362 |
| Beta: age                        |         -7.449 |      | 75,441.043 |   0.971 |   -147,892.897 |    147,877.998 |
| Beta: borough_name_Bronx         | -3,107,734.887 | ***  | 74,462.161 |   0.000 | -3,253,701.453 | -2,961,768.320 |
| Beta: borough_name_Brooklyn      | -2,659,313.821 | ***  | 74,276.127 |   0.000 | -2,804,915.709 | -2,513,711.933 |
| Beta: borough_name_Queens        | -2,870,994.746 | ***  | 75,420.358 |   0.000 | -3,018,839.645 | -2,723,149.847 |
| Beta: borough_name_Staten_Island | -3,023,379.363 | **

In [55]:
# 0: Bronx, 1: Brooklyn, 2: Manhattan, 3: Queens, 4: Staten Island
dummy_cols, ref_category = add_dummy_variables("borough_name", 0)

Reference category (dummy omitted): Bronx


In [56]:
dtrain.select(['borough_name'] + dummy_cols).show()

+------------+---------------------+----------------------+-------------------+--------------------------+
|borough_name|borough_name_Brooklyn|borough_name_Manhattan|borough_name_Queens|borough_name_Staten_Island|
+------------+---------------------+----------------------+-------------------+--------------------------+
|       Bronx|                    0|                     0|                  0|                         0|
|       Bronx|                    0|                     0|                  0|                         0|
|    Brooklyn|                    1|                     0|                  0|                         0|
|    Brooklyn|                    1|                     0|                  0|                         0|
|    Brooklyn|                    1|                     0|                  0|                         0|
|    Brooklyn|                    1|                     0|                  0|                         0|
|    Brooklyn|                    1| 

In [57]:
conti_cols = ["gross_square_feet", "age"]
assembler_predictors = conti_cols + dummy_cols

assembler3 = VectorAssembler(
    inputCols = assembler_predictors,
    outputCol = "predictors"
)
dtrain3 = assembler3.transform(dtrain)
dtest3  = assembler3.transform(dtest)
model3 = LinearRegression(featuresCol="predictors", labelCol="sale_price").fit(dtrain3)
dtest3 = model3.transform(dtest3)

# For model3 and assembler3:
print(regression_table(model3, assembler3))

+----------------------------------+---------------+------+------------+---------+---------------+---------------+
| Metric                           |         Value | Sig. | Std. Error | p-value |  95% CI Lower |  95% CI Upper |
+----------------------------------+---------------+------+------------+---------+---------------+---------------+
| Beta: gross_square_feet          |       391.989 | ***  |    204.243 |   0.000 |        -8.384 |       792.362 |
| Beta: age                        |        -7.449 |      | 19,920.723 |   0.971 |   -39,057.612 |    39,042.713 |
| Beta: borough_name_Brooklyn      |   448,421.065 | ***  | 75,441.043 |   0.000 |   300,535.618 |   596,306.513 |
| Beta: borough_name_Manhattan     | 3,107,734.887 | ***  | 17,740.096 |   0.000 | 3,072,959.359 | 3,142,510.414 |
| Beta: borough_name_Queens        |   236,740.140 | ***  | 19,926.890 |   0.000 |   197,677.888 |   275,802.393 |
| Beta: borough_name_Staten_Island |    84,355.524 | ***  | 27,918.613 |   0.000